# Accessing APIs

## Introduction

This notebook will guide you through the fundamentals of accessing and working with web APIs using Python. APIs (Application Programming Interfaces) are essential tools for modern data science, allowing you to access real-time data from countless sources including government databases, weather services, financial markets, and research institutions.

We'll work with real-world APIs to gather data about Rwanda, demonstrating practical techniques you can apply to any data science project. You'll learn to handle different types of APIs, from completely free services to those requiring authentication, while following professional best practices.

### Why This Matters

APIs are everywhere in modern data science. Whether you're building economic analysis dashboards, environmental monitoring systems, or social research platforms, the ability to programmatically access external data sources is crucial. This workshop teaches you not just how to make API calls, but how to do so reliably, ethically, and professionally.

## Learning Outcomes

By the end of this workshop, you will be able to:

1. **Make successful API requests** using Python's `requests` library, handling different authentication methods and response formats
2. **Process and analyze JSON data** returned by APIs, converting it into pandas DataFrames for further analysis
3. **Implement professional practices** including error handling, rate limiting, retry logic, and proper credential management
4. **Work with multiple API types** from free government APIs to commercial services requiring API keys
5. **Combine data from multiple sources** to create comprehensive datasets for analysis

---


## World bank APIs

The World Bank API provides free access to development data from around the world. It's an ideal starting point for learning API consumption because it requires no authentication and provides reliable, well-structured data.

### API Details

#### Authentication
- **No API key required**
- **No registration needed**
- **Completely free to use**
- No rate limiting restrictions (reasonable use expected)

#### Base URL
```
https://api.worldbank.org/v2/
```

#### Required Python Packages
No special packages required - uses standard HTTP requests.

####  Request Structure
The World Bank API follows a RESTful structure with the following pattern:
```
https://api.worldbank.org/v2/{endpoint}?{parameters}
```

####  Common Endpoints
- **Countries**: `/country` - List all countries
- **Indicators**: `/indicator` - List all available indicators  
- **Data**: `/country/{country_code}/indicator/{indicator_code}` - Get specific data

#### Required Parameters
- **format**: Must be set to `json` (API defaults to XML)
  ```python
  params = {'format': 'json'}
  ```

#### Optional Parameters
- **date**: Specify year range (e.g., `2015:2023` or `2020`)
- **per_page**: Number of results per page (default: 50, max: 32,500)
- **page**: Page number for pagination
- **source**: Filter by data source

#### Example Request
```python
import requests

url = "https://api.worldbank.org/v2/country/rw/indicator/SP.POP.TOTL"
params = {
    'format': 'json',
    'date': '2015:2023',
    'per_page': 100
}

response = requests.get(url, params=params)
data = response.json()
```

#### Response Structure
The API returns an array with two elements:
1. **Metadata** (index 0): Information about the request and pagination
2. **Data** (index 1): Array of actual data records

Each data record contains:
- `indicator`: Indicator information
- `country`: Country information  
- `countryiso3code`: 3-letter country code
- `date`: Year
- `value`: Data value
- `unit`: Unit of measurement
- `decimal`: Number of decimal places

#### Popular Indicators for Rwanda
- `SP.POP.TOTL` - Population, total
- `NY.GDP.MKTP.CD` - GDP (current US$)
- `NY.GDP.PCAP.CD` - GDP per capita (current US$)
- `SI.POV.DDAY` - Poverty headcount ratio at $2.15 a day
- `SP.DYN.LE00.IN` - Life expectancy at birth
- `SE.ADT.LITR.ZS` - Literacy rate, adult total
- `IT.NET.USER.ZS` - Individuals using the Internet

#### Country Codes
- Rwanda: `rw`
- Uganda: `ug`  
- Tanzania: `tz`
- Kenya: `ke`
- Democratic Republic of Congo: `cd`

### Error Handling
The API returns HTTP status codes:
- `200` - Success
- `400` - Bad request (invalid parameters)
- `404` - Not found (invalid country/indicator code)

### Data Quality Notes
- Some indicators may have missing data for certain years
- Always check for `null` values in the response
- Historical data availability varies by indicator and country
- Most recent data may be 1-2 years behind current year

### Complete Documentation
**Official API Documentation**: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392

**Additional Resources**:
- **Indicator Catalog**: https://datacatalog.worldbank.org/
- **Country Classifications**: https://datahelpdesk.worldbank.org/knowledgebase/articles/906519
- **API Terms of Use**: https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets

### Best Practices
- Always include `format=json` parameter
- Use specific date ranges to limit response size
- Handle missing/null values in your data processing
- Cache responses for repeated analysis to reduce server load
- Be respectful with request frequency (1-2 requests per second maximum)

## Explore the API

In [11]:
import requests
!pip install fastapi uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import json
import numpy as np
import pandas as pd

In [14]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, Kigali!"}
import time


In [ ]:
def get_multiple_wb_indicators():
    """
    Fetch multiple economic indicators for Rwanda
    """
    base_url = "https://api.worldbank.org/v2"
    
    # Different indicators
    indicators = {
        'GDP_GROWTH': 'NY.GDP.MKTP.KD.ZG',  # GDP growth
        'POVERTY_RATE': 'SI.POV.DDAY',      # Poverty headcount
        'LIFE_EXPECTANCY': 'SP.DYN.LE00.IN', # Life expectancy
        'INTERNET_USERS': 'IT.NET.USER.ZS'   # Internet users %
    }
    
    all_data = {}
    
    for name, indicator_code in indicators.items():
        url = f"{base_url}/country/rw/indicator/{indicator_code}"
        params = {
            'format': 'json',
            'date': '2015:2022',
            'per_page': 50
        }
        
        print(f"Fetching {name}...")
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if len(data) > 1 and data[1]:
                # Convert to DataFrame
                df = pd.DataFrame(data[1])
                df_clean = df[['date', 'value']].dropna()
                df_clean.columns = ['year', name.lower()]
                all_data[name] = df_clean
        
        # Be respectful - add small delay
        time.sleep(0.5)
    
    return all_data

# Get multiple indicators
indicators_data = get_multiple_wb_indicators()

# Display results
for name, df in indicators_data.items():
    print(f"\n{name}:")
    print(df.head())



In [37]:
import requests

base_url = "https://data360.worldbank.org/en/api#/Data/get_data360_data"
response = requests.get(base_url)

print(response.status_code)  # check if request worked
print(response.text)         # raw response
base_url


200

<!DOCTYPE HTML>
<html lang="en">
    <head>
              
    <meta charset="UTF-8"/>
    
        <title>API | World Bank Data360</title>
        
            <script defer="defer" type="text/javascript" src="https://rum.hlx.page/.rum/@adobe/helix-rum-js@%5E2/dist/rum-standalone.js" data-routing="env=stage,tier=publish,ams=World Bank"></script>
<link rel="preconnect" href="https://assets.adobedtm.com/" crossorigin=""/>
<link rel="preconnect" href="https://s7d1.scene7.com/" crossorigin=""/>
<link rel="preconnect" href="https://fonts.gstatic.com/" crossorigin=""/>
<link rel="preconnect" href="https://data360api.worldbank.org/" crossorigin=""/>
<link rel="preconnect" href="https://extdataportal.worldbank.org/" crossorigin=""/>
<link rel="preconnect" href="https://webapi.worldbank.org/" crossorigin=""/>

        
    
    
 	
	
    <meta name="template" content="data360-static-page"/>
    <meta name="viewport" content="width=device-width, initial-scale=1"/>
    

    
    
   <meta 

'https://data360.worldbank.org/en/api#/Data/get_data360_data'

In [38]:
endpoint_url = 'data'
full_url = base_url + endpoint_url 
response = requests.get(full_url)
print(response.status_code)  # check if request worked
full_url

200


'https://data360.worldbank.org/en/api#/Data/get_data360_datadata'

In [ ]:
dict_obj_from_json = response.json()
type(dict_obj_from_json)
print(dict_obj_from_json),type(dict_obj_from_json)

In [39]:
base_url = 'https://raw.githubusercontent.com/worldbank/open-api-specs/refs/heads/main/Data360%20Open_API.json'
response = requests.get(base_url)

print(response.status_code)  # check if request worked
print(response.text)         # raw response
base_url

200
{
  "openapi": "3.0.1",
  "info": {
    "title": "Data360 API",
    "description": "This API provides users access to metadata and data files for datasets within the World Bank's Data360 Data.",
    "contact": {
      "name": "Data360 Data Support",
      "email": "data@worldbank.org"
    },
    "license": {
      "name": "Creative Commons 4.0 by Attribution",
      "url": "https://creativecommons.org/licenses/by/4.0"
    },
    "version": "v1"
  },
  "servers": [
    {
      "url": "https://data360api.worldbank.org",
      "description": "API root URL"
    }
  ],
  "tags": [
    {
      "name": "Search",
      "description": "Endpoints to search Data360."
    },
    {
      "name": "Data",
      "description": "Endpoints to retrieve data."
    },
    {
      "name": "Metadata",
      "description": "Endpoints to retrieve metadata."
    },
    {
      "name": "Other",
      "description": "Endpoints providing other functionalities."
    }
  ],
  "paths": {
    "/data360/data": {
  

'https://raw.githubusercontent.com/worldbank/open-api-specs/refs/heads/main/Data360%20Open_API.json'

In [40]:
dict_obj_from_json = response.json()
type(dict_obj_from_json)
print(dict_obj_from_json),type(dict_obj_from_json)

{'openapi': '3.0.1', 'info': {'title': 'Data360 API', 'description': "This API provides users access to metadata and data files for datasets within the World Bank's Data360 Data.", 'contact': {'name': 'Data360 Data Support', 'email': 'data@worldbank.org'}, 'license': {'name': 'Creative Commons 4.0 by Attribution', 'url': 'https://creativecommons.org/licenses/by/4.0'}, 'version': 'v1'}, 'servers': [{'url': 'https://data360api.worldbank.org', 'description': 'API root URL'}], 'tags': [{'name': 'Search', 'description': 'Endpoints to search Data360.'}, {'name': 'Data', 'description': 'Endpoints to retrieve data.'}, {'name': 'Metadata', 'description': 'Endpoints to retrieve metadata.'}, {'name': 'Other', 'description': 'Endpoints providing other functionalities.'}], 'paths': {'/data360/data': {'get': {'summary': 'Data based on provided filters', 'description': 'Retrieves dataset or indicators data based on a range of query filters.', 'tags': ['Data'], 'parameters': [{'$ref': '#/components/pa

(None, dict)

In [13]:
import pandas as pd

# JSON string
json_data = '{"name":["Karenzo","Evariste"], "age":[25,30]}'

# Convert to DataFrame
df = pd.read_json(json_data)

print(df)


       name  age
0   Karenzo   25
1  Evariste   30


C:\Users\evariste.manirumva\AppData\Local\Temp\ipykernel_27048\3582836255.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data)
